<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/14_Customizing_Model_Fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Device setting

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Data load

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize

In [4]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Model

In [5]:
model = keras.Sequential(
    [
     layers.Input(shape=(28, 28, 1)),
     layers.Conv2D(64, 3, padding='same'),
     layers.ReLU(),
     layers.Conv2D(128, 3, padding='same'),
     layers.ReLU(),
     layers.Flatten(),
     layers.Dense(10)
    ],
    name='model',
)

In [6]:
class CustomFit(keras.Model):
  def __init__(self, model):
    super(CustomFit, self).__init__()
    self.model = model
  
  def compile(self, optimizer, loss):
    super(CustomFit, self).compile()
    self.optimizer = optimizer
    self.loss = loss

  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x, training = True)
      loss = self.loss(y, y_pred)
    
    training_vars = self.trainable_variables
    gradients = tape.gradient(loss, training_vars)

    self.optimizer.apply_gradients(zip(gradients, training_vars))
    acc_metric.update_state(y, y_pred)

    return {'loss' : loss, 'accuracy' : acc_metric.result()}
  
  def test_step(self, data):
    x, y = data

    y_pred = self.model(x, training=False)
    loss = self.loss(y, y_pred)
    acc_metric.update_state(y, y_pred)

    return {'loss': loss, 'accuracy':acc_metric.result()}


acc_metric = keras.metrics.SparseCategoricalCrossentropy(name='accuracy')
training = CustomFit(model)
training.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)
training.fit(x_train, y_train, batch_size=32, epochs=2)
training.evaluate(x_test, y_test, batch_size=32)


Epoch 1/2
1875/1875 [==============================] - 24s 12ms/step - loss: 0.1139 - accuracy: 0.6790
Epoch 2/2
313/313 [==============================] - 2s 6ms/step - loss: 0.0551 - accuracy: 0.2556


[0.2529182732105255, 0.0005138777196407318]